# Demo - age and gender recognition via REST API

This notebook presents how to use OpenVINO Model Server to execute inference requests over REST API interface.
The demo is using a pretrained model from [open_model_zoo](https://github.com/opencv/open_model_zoo) repository.

### Download the pretrained model for age and gender recognition

In [1]:
!git clone https://github.com/opencv/open_model_zoo.git

Cloning into 'open_model_zoo'...
remote: Enumerating objects: 307, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 4080 (delta 127), reused 294 (delta 126), pack-reused 3773
Receiving objects: 100% (4080/4080), 70.04 MiB | 2.53 MiB/s, done.
Resolving deltas: 100% (2097/2097), done.
Checking connectivity... done.


In [2]:
!cd open_model_zoo/model_downloader; python downloader.py --name age-gender-recognition-retail-0013


###############|| Downloading topologies ||###############

========= Downloading /private/tmp/open_model_zoo/model_downloader/Retail/object_attributes/age_gender/dldt/age-gender-recognition-retail-0013.xml
... 100%, 14 KB, 143 KB/s, 0 seconds passed

========= Downloading /private/tmp/open_model_zoo/model_downloader/Retail/object_attributes/age_gender/dldt/age-gender-recognition-retail-0013.bin
... 100%, 8351 KB, 1571 KB/s, 5 seconds passed


###############|| Post processing ||###############



### Build the docker image with OVMS component

In [3]:
!git clone https://github.com/IntelAI/OpenVINO-model-server

Cloning into 'OpenVINO-model-server'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 1020 (delta 36), reused 33 (delta 15), pack-reused 954
Receiving objects: 100% (1020/1020), 3.08 MiB | 1.99 MiB/s, done.
Resolving deltas: 100% (613/613), done.
Checking connectivity... done.


In [4]:
!cd OpenVINO-model-server ; make docker_build_src_intelpython

Building docker image
docker build -f Dockerfile_intelpython --build-arg http_proxy=<http_proxy> --build-arg https_proxy=<https_proxy> -t ie-serving-py:latest .
Sending build context to Docker daemon   10.9MB
Step 1/31 : FROM intelpython/intelpython3_core as DEV
 ---> 3fbe3dacb980
Step 2/31 : RUN apt-get update && apt-get install -y             autoconf             automake             build-essential             ca-certificates             curl             git             gstreamer1.0-plugins-base             libavcodec-dev             libavformat-dev             libboost-regex-dev             libcairo2-dev             libgfortran3             libglib2.0-dev             libgstreamer1.0-0             libgtk2.0-dev             libopenblas-dev             libpango1.0-dev             libpng-dev             libssl-dev             libswscale-dev             libtool             libusb-1.0-0-dev             pkg-config             unzip             vim             wget
 ---> Using cache
 ---> 

### Start OVMS docker container with downloaded model

In [5]:
!export DOCKER_IMAGE_NAME=ie-serving-py;\
docker run -d --rm -v ${PWD}/open_model_zoo/model_downloader/Retail/object_attributes/age_gender/dldt:/opt/ml/age_gender/1 -p 9000:9000 -p 8000:8000 ${DOCKER_IMAGE_NAME} /ie-serving-py/start_server.sh ie_serving model --model_path /opt/ml/age_gender --model_name age_gender --port 9000 --rest_port 8000

281614d378c9811c7b83092b1cb8db8763e7a7857dd0029b6f30f9e5bf539b51


### Download sample image

In [6]:
!wget https://raw.githubusercontent.com/opencv/open_model_zoo/master/intel_models/age-gender-recognition-retail-0013/description/age-gender-recognition-retail-0001.jpg

--2019-06-28 14:39:50--  https://raw.githubusercontent.com/opencv/open_model_zoo/master/intel_models/age-gender-recognition-retail-0013/description/age-gender-recognition-retail-0001.jpg
Resolving proxy-chain.intel.com... 163.33.7.140
Connecting to proxy-chain.intel.com|163.33.7.140|:912... connected.
Proxy request sent, awaiting response... 200 OK
Length: 2436 (2.4K) [image/jpeg]
Saving to: 'age-gender-recognition-retail-0001.jpg'

age-gender-recognit 100%[===================>]   2.38K  --.-KB/s    in 0s      

2019-06-28 14:39:50 (17.9 MB/s) - 'age-gender-recognition-retail-0001.jpg' saved [2436/2436]



![age-gender-recognition-retail-0001.jpg](age-gender-recognition-retail-0001.jpg)

### Format the json request

In [7]:
import cv2
import numpy as np
import json


def getJpeg(path, size):

    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # retrived array has BGR format and 0-255 normalization
    img = cv2.resize(img, (size, size))
    img = img.transpose(2,0,1).reshape(1,3,size,size)
    print(path, img.shape, "; data range:",np.amin(img),":",np.amax(img))
    return img

my_image = getJpeg('age-gender-recognition-retail-0001.jpg',62)


data_obj = {'inputs':  my_image.tolist()}
data_json = json.dumps(data_obj)
print(data_json)

age-gender-recognition-retail-0001.jpg (1, 3, 62, 62) ; data range: 0 : 239
{"inputs": [[[[232, 226, 234, 232, 228, 231, 221, 198, 174, 115, 89, 73, 65, 57, 56, 58, 53, 49, 45, 42, 41, 31, 27, 24, 32, 42, 51, 53, 43, 42, 34, 38, 33, 30, 26, 23, 24, 24, 25, 23, 25, 26, 25, 24, 21, 21, 21, 21, 24, 19, 26, 24, 35, 40, 56, 63, 90, 127, 214, 197, 205, 195], [226, 233, 234, 229, 230, 228, 208, 187, 135, 82, 66, 49, 43, 65, 56, 60, 52, 48, 41, 37, 34, 31, 26, 28, 37, 43, 53, 50, 44, 37, 32, 32, 29, 28, 25, 22, 23, 23, 25, 25, 25, 26, 24, 23, 21, 20, 20, 20, 20, 19, 25, 26, 27, 31, 44, 59, 81, 151, 207, 200, 199, 196], [230, 229, 234, 234, 222, 215, 184, 122, 85, 64, 49, 56, 61, 62, 55, 48, 52, 47, 39, 33, 26, 27, 22, 28, 44, 47, 59, 41, 44, 33, 35, 27, 27, 26, 24, 22, 21, 21, 22, 23, 22, 23, 23, 22, 22, 23, 21, 21, 20, 21, 21, 26, 21, 25, 35, 60, 94, 170, 204, 195, 196, 195], [233, 231, 227, 226, 215, 162, 136, 77, 59, 66, 60, 57, 60, 52, 55, 52, 44, 37, 35, 30, 25, 27, 25, 37, 51, 44, 37, 44

### Run the inference request to OVMS REST API endpoint

In [8]:
import requests


result = requests.post("http://localhost:8000/v1/models/age_gender:predict", data=data_json)
result_dict = json.loads(result.text)
print(result_dict)

{'outputs': {'age_conv3': [[[[0.2519038915634155]]]], 'prob': [[[[0.9874807000160217]], [[0.012519358657300472]]]]}}


Outputs
name: "age_conv3", shape: [1, 1, 1, 1] - Estimated age divided by 100.
name: "prob", shape: [1, 2, 1, 1] - Softmax output across 2 type classes [female, male]